# Quantum Walk on Line Topology

## What is Quantum Walk?

A quantum (random) walk is the quantum counterpart of a classical random walk, where the walker (particle) follows a probabilistic path. Quantum random walks can outperform classical random walks in certain computational tasks, such as search algorithms and solving specific mathematical problems more efficiently. They form the basis for some of the most promising quantum algorithms, providing potential speedups for problems that are intractable for classical computers.

## Objective:

The goal of this quantum program is to simulate a quantum random walk on a line topology with 16 nodes.

In a line topology, the nodes are arranged sequentially, and each node is connected to its immediate neighbors. For example, consider a line of 16 nodes numbered from 0 to 15. In this walk, the walker would move from its current node to an adjacent node (either left or right) with equal probability.

Image source: Womanium + Quantum AI

Let's start by importing our required packages and setting the number of qubits as 4 ($log_2(16)$)

In [1]:
from classiq import *
from classiq.qmod.symbolic import logical_or
from classiq.execution import ExecutionPreferences

In [3]:
size = 4 # Number of qubits to represent the vertices (log2 of 16 nodes)
num_nodes = 2**size # Total number of nodes (16 nodes)

The algorithm will include two major steps:
- Coin Operator: determines the walker's movement probabilities
- Shift Operator: execute actual movement

Let's look at each operator one by one.

## Coin Operator

The coin operator, a quantum analog of flipping a coin, which determines the walker's movement probabilities. The coin operator transforms the walker's state by assigning amplitudes to move left or right from each node. For instance, if the walker is at node 0, the coin operator will set the probability to move to node 1. If the walker is at node 15, it will set the probability to move to node 14. For nodes in between, the probabilities are distributed evenly to move either left or right.

For our process, we'll setup the probability based on above discussed idea. We, then, use diffuser oracle to to amplify the probabilities of adjacent nodes.

In [4]:
# Function to prepare a qubit in the |-> state
@qfunc
def prepare_minus(x: QBit):
    """
    Prepares a qubit in the |-> state.
    
    Args:
        x (QBit): The qubit to prepare.
    """
    X(x)
    H(x)

@qfunc
def diffuzer_oracle(aux: Output[QNum],x:QNum):
    """
    Implements a diffuser oracle for Grover's algorithm.
    
    Args:
        aux (Output[QNum]): Auxiliary qubit.
        x (QNum): QNum representing the node to check.
    """
    aux^=(x!=0)

@qfunc
def zero_diffuzer(x: QNum):
    """
    Implements the zero diffuser for the quantum walk.
    
    Args:
        x (QNum): QNum representing the current node.
    """
    aux = QNum('aux')
    allocate(1,aux)
    within_apply(compute=lambda: prepare_minus(aux),
              action=lambda: diffuzer_oracle)

def C_iteration(i:int,vertices: QNum, adjacent_vertices:QNum):
    """
    Defines the coin operation for the quantum walk.
    
    Args:
        i (int): Current node index.
        vertices (QNum): QNum representing the vertices.
        adjacent_vertices (QNum): QNum representing the adjacent vertices.
    """
    prob = [0]*num_nodes  # Initialize probability vector for 16 nodes
    if i == 0:
        prob[i + 1] = 1.0  # If at the first node, move to the right node (node 1)
    elif i == num_nodes - 1:
        prob[i - 1] = 1.0  # If at the last node, move to the left node (node 14)
    else:
        prob[i - 1] = 0.5  # Probability of moving to the left node
        prob[i + 1] = 0.5  # Probability of moving to the right node
    print(f'Node={i}, prob vec ={prob}')
    control(ctrl=vertices==i,
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
              action= lambda: zero_diffuzer(adjacent_vertices)))

@qfunc 
def C_operator(vertices:QNum, adjacent_vertices: QNum):
    """
    Applies the coin operator to all vertices.
    
    Args:
        vertices (QNum): QNum representing the vertices.
        adjacent_vertices (QNum): QNum representing the adjacent vertices.
    """
    for i in range(num_nodes):
        C_iteration(i,vertices,adjacent_vertices)


The coin operator prepares an adjacent vertex state, which will be utilized by the shift operator. Each node index will undergo the coin operation to set its movement probabilities.

## Shift Operator

The shift operator moves the walker from the current node to an adjacent node based on the probability amplitude assigned by the coin operator. For our line topology, this means moving to the left or right node. 

Let's define the shift operation.

In [7]:
# Swap control qubits
@qfunc
def Swap_control(vertices: QNum, adj_vertices: QNum):
    """
    Swaps the control qubits representing the current node and the adjacent node.
    
    Args:
        vertices (QNum): QNum representing the vertices.
        adj_vertices (QNum): QNum representing the adjacent vertices.
    """
    for j in range(len(vertices)):
        Swap(vertices[j], adj_vertices[j])

@qfunc
def shift_operation(vertices:QNum, adj_vertices:QNum):
    """
    Applies the shift operation to all vertices.
    
    Args:
        vertices (QNum): QNum representing the vertices.
        adj_vertices (QNum): QNum representing the adjacent vertices.
    """
    for i in range(num_nodes):
        control(ctrl=vertices==i, operand= lambda: Swap_control(vertices, adj_vertices))


The shift operator effectively swaps the walker's position with an adjacent node.

## Complete Quantum Walk

Combining the coin and shift operators, we can now define the quantum walk. We will repeat the coin and shift operations multiple times to simulate the walk over a given number of steps.

In [8]:
@qfunc
def quantum_walk(steps: int):
    """
    Executes a quantum walk on the line topology for a specified number of steps.
    
    Args:
        steps (int): Number of steps to execute the quantum walk.
    """
    vertices = QNum('vertices', size)
    adjacent_vertices = QNum('adjacent_vertices', size)
    allocate(size, vertices)
    allocate(size, adjacent_vertices)

    # Initialize the walker to the starting node (node 0)
    vertices ^= 0

    for _ in range(steps):
        C_operator(vertices, adjacent_vertices)
        shift_operation(vertices, adjacent_vertices)
    return vertices

We have defined the quantum walk function which executes the coin and shift operations for a given number of steps. Let's now create the Classiq Model and visualize it.

In [9]:
# Define the quantum walk model
model = ClassiqModel(quantum_walk(steps=10), ExecutionPreferences(visualize=True, circuit_compilation_target='quantinuum_hqs_linq_cloud'))

# Visualize the model
model.visualize()

The quantum walk model is now defined and visualized. The visualization shows the quantum circuit representing the quantum walk on a line topology with 16 nodes.